### Description: NEW
Refine new cohort4 to only include CSNs that are not already in the original cohort
- New cohort: shc_core_2021, from 04/2020 - 2021
- Original old cohort: shc_core, 2015 - 03/2020

However, some CSNs in the new cohort are also in the old cohort, but with different anon_id and admit time
- Remove these overlapping CSNs. This is clean if use both cohort4 from old and new data
- When using Demographics/HW data later in 6.8 notebook, be aware that there are a few overlapped CSNs. This is due to cohort2 was used to get demo/HW data. These overlapped CSNs were actually removed from cohort3 already. They are removed from new cohort4 again when merging with the old demo/HW data to prevent further issues. But this `cohort4` remain intact.

**Input:**
- `1_4_cohort.csv` (original cohort4)
- `6_7_0_cohort4` (new cohort4 with labels)


**Output:**
- `6_7_cohort4` size 60,464. This is the *final cohort* combing 2015 - 03/2020 (43,980) and 04/2020 - 2021 (16,484) data


In [1]:
library(data.table)
library(tidyverse)
library(lubridate)
# library(Matrix)
# library(slam)
# library(bit64)
# library(mtools) for one hot coder, not available on Nero or use caret or tidyr
options(repr.matrix.max.rows=200, repr.matrix.max.cols=30)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [2]:
datadir = "../../DataTD"
datadir6 = "../../DataTD/validation"
valdir = "../../OutputTD/6_validation"
cohortdir = "../../OutputTD/1_cohort"
featuredir = "../../OutputTD/2_features"

In [3]:
# old cohort up to 03/2020
cohort0 <- read.csv(file.path(cohortdir,  '1_4_cohort.csv'))
nrow(cohort0) #43980

# new cohort from 04/2020 - 2021
cohort6 <- read.csv(file.path(valdir,  '6_7_0_cohort4.csv'))
nrow(cohort6) #16700

[1] 43980

[1] 16700

In [4]:
# check overlapping cohort
length(setdiff(cohort0$pat_enc_csn_id_coded, cohort6$pat_enc_csn_id_coded))
length(setdiff(cohort6$pat_enc_csn_id_coded, cohort0$pat_enc_csn_id_coded)) 

[1] 43764

[1] 16484

In [5]:
# csn in cohort4 but not in 0
csn_new <- setdiff(cohort6$pat_enc_csn_id_coded, cohort0$pat_enc_csn_id_coded)
head(csn_new)

[1] 131295313275 131295018112 131288774622 131302440087 131305861173
[6] 131308681280

In [6]:
# merge with unique csn
cohort6 <- cohort6 %>% filter(pat_enc_csn_id_coded %in% csn_new)
nrow(cohort6)
length(unique(cohort6$pat_enc_csn_id_coded))

[1] 16484

[1] 16484

In [7]:
cohort <- bind_rows(cohort0, cohort6)
nrow(cohort)
nrow(cohort %>% select(anon_id) %>% distinct()) # 60464
nrow(cohort %>% select(pat_enc_csn_id_coded) %>% distinct()) # 60464
nrow(cohort %>% select(anon_id, pat_enc_csn_id_coded) %>% distinct()) # 60464
head(cohort)

[1] 60464

[1] 43026

[1] 60464

[1] 60464

,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label_max24,label_24hr_recent,admit_label,has_admit_label,died_within_24hrs,death_24hr_max_label,death_24hr_recent_label,first_label,first_label_minutes_since_admit,acute_to_critical_label_recent,critical_to_acute_label_recent,acute_to_critical_label_max,critical_to_acute_label_max
,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>
1,JCd97296,131176042095,18290644,2016-02-06 22:31:00+00:00,0,0,NA,0,0,0,0,0,1325,0,0,0,0
2,JCcdc7e1,131064611420,13865299,2015-01-15 21:16:00+00:00,1,1,1,1,0,1,1,1,0,0,0,0,0
3,JCe3e5f4,131072326078,14296997,2015-01-28 11:12:00+00:00,1,1,1,1,0,1,1,1,0,0,0,0,0
4,JCdcfce9,131178712824,18633398,2016-03-04 17:01:00+00:00,1,1,1,1,0,1,1,1,0,0,0,0,0
5,JCdaaaa6,131211945620,22773101,2016-12-07 22:17:00+00:00,0,0,0,1,0,0,0,0,0,0,0,0,0
6,JCe8840f,131264906504,34995073,2019-02-14 22:22:00+00:00,0,0,0,1,0,0,0,0,0,0,0,0,0


In [8]:
summary(cohort)

   anon_id          pat_enc_csn_id_coded inpatient_data_id_coded
 Length:60464       Min.   :1.311e+11    Min.   :13616753       
 Class :character   1st Qu.:1.312e+11    1st Qu.:22754408       
 Mode  :character   Median :1.313e+11    Median :32943195       
                    Mean   :1.312e+11    Mean   :37505576       
                    3rd Qu.:1.313e+11    3rd Qu.:54331357       
                    Max.   :1.313e+11    Max.   :73231043       
                                                                
  admit_time         label_max24     label_24hr_recent  admit_label    
 Length:60464       Min.   :0.0000   Min.   :0.0000    Min.   :0.0000  
 Class :character   1st Qu.:0.0000   1st Qu.:0.0000    1st Qu.:0.0000  
 Mode  :character   Median :0.0000   Median :0.0000    Median :0.0000  
                    Mean   :0.1352   Mean   :0.0949    Mean   :0.1086  
                    3rd Qu.:0.0000   3rd Qu.:0.0000    3rd Qu.:0.0000  
                    Max.   :1.0000   Max.   :1.0

In [9]:
summary(cohort6)

   anon_id          pat_enc_csn_id_coded inpatient_data_id_coded
 Length:16484       Min.   :1.313e+11    Min.   :51964995       
 Class :character   1st Qu.:1.313e+11    1st Qu.:57273662       
 Mode  :character   Median :1.313e+11    Median :62306884       
                    Mean   :1.313e+11    Mean   :62534719       
                    3rd Qu.:1.313e+11    3rd Qu.:67813306       
                    Max.   :1.313e+11    Max.   :73231043       
                                                                
  admit_time         label_max24     label_24hr_recent  admit_label   
 Length:16484       Min.   :0.0000   Min.   :0.00000   Min.   :0.000  
 Class :character   1st Qu.:0.0000   1st Qu.:0.00000   1st Qu.:0.000  
 Mode  :character   Median :0.0000   Median :0.00000   Median :0.000  
                    Mean   :0.1349   Mean   :0.09597   Mean   :0.105  
                    3rd Qu.:0.0000   3rd Qu.:0.00000   3rd Qu.:0.000  
                    Max.   :1.0000   Max.   :1.00000  

In [12]:
nrow(cohort)
nrow(cohort6)

[1] 60464

[1] 16484

In [14]:
write.csv(cohort6, file = file.path(valdir, "6_7_cohort4.csv"), row.names=FALSE) 
write.csv(cohort, file = file.path(valdir, "6_7_cohort4_all.csv"), row.names=FALSE) 